# Play Time Genres
- play_time_genres toma como parámetro un género particular, y retorna el año con mayor tiempo jugado para dicho genero.

- Para evitar consultas lentas, crearemos una tabla auxiliar para realizar esta consulta. 


In [36]:
import pandas as pd
import os 

path_user = os.path.join('..','data','clear','users_items.csv.gz')
path_steam_games = os.path.join('..','data','clear','steam_games.csv.gz')

## Carga de data

In [37]:
## Cargamos la data necesaria para solucionar este problema.
steam_columns = ['id','genres', 'release_year']
steam_games = pd.read_csv(path_steam_games, usecols= steam_columns)

## renombraremos la columna id por steam_id
steam_games.rename(columns={'id':'steam_id'}, inplace=True)

user_item_cols = ['item_id', 'item_name', 'playtime_forever']
user_items = pd.read_csv(path_user, usecols = user_item_cols)

In [38]:
steam_games.head(3)

,genres,steam_id,release_year
0,"Strategy, Action, Indie, Casual, Simulation",761140.0,2018
1,"Free to Play, Strategy, Indie, RPG, Card Game,...",643980.0,2018
2,"Free to Play, Simulation, Sports, Casual, Indi...",670290.0,2017


In [39]:
user_items.head(3)

,playtime_forever,item_id,item_name
0,6.0,10.0,Counter-strike
1,0.0,20.0,Team fortress classic
2,7.0,30.0,Day of defeat


## Creación de tabla consulta
- En esta sección combinaremos steam_games  con user_items, por medio de sus ids.
Luego agruparemos por año de y grupo de genero comun, para calcular la suma de tiempos jugados por dicho genero.

In [40]:
# realizamos un inner join, ya que no nos interan juegos que no se encuentren en steam y tampoco nos interesa juegos de steam que no tenga información de usuarios que jugaron en ese juego.
steam_users = steam_games.merge(user_items,
                                left_on='steam_id', 
                                right_on= 'item_id',
                                how='inner')


In [41]:
steam_users.sample(5)

,genres,steam_id,release_year,playtime_forever,item_id,item_name
1670713,"Free to Play, Fishing, Simulation, Multiplayer...",380600.0,2015,0.0,380600.0,Fishing planet
1109108,"Adventure, Point & Click, Family Friendly, Cas...",283920.0,1992,63.0,283920.0,Putt-putt joins the parade
3932231,"Music, Rhythm, Indie, Casual, Music-Based Proc...",12900.0,2008,1958.0,12900.0,Audiosurf
3681133,"FPS, Action, Singleplayer, Story Rich, Atmosph...",8850.0,2010,3694.0,8850.0,Bioshock 2
3155543,"Turn-Based Strategy, Tactical, Strategy, Sci-f...",200510.0,2012,2117.0,200510.0,Xcom: enemy unknown


- Por cada año exinten muchos generos, y cada grupo diferente de año va a tener el mismo tiempo de juego. es por esto que agruparemos por la columna genero. Luego cada elemeno va a tener una lista de generos que van a tener el mismo tiempo de juego. Para que esta consulta sea considerada apropiada veremos que existe una cantidad considerablemente pequeña de grupos de generos.

In [42]:
len(steam_users['genres'].unique())

7679

In [69]:
steam_users

,genres,steam_id,release_year,playtime_forever,item_id
0,"Racing, Action, Classic, Indie, Gore, 1990's, ...",282010.0,1997,5.0,282010.0
1,"Racing, Action, Classic, Indie, Gore, 1990's, ...",282010.0,1997,0.0,282010.0
2,"Racing, Action, Classic, Indie, Gore, 1990's, ...",282010.0,1997,0.0,282010.0
3,"Racing, Action, Classic, Indie, Gore, 1990's, ...",282010.0,1997,0.0,282010.0
4,"Racing, Action, Classic, Indie, Gore, 1990's, ...",282010.0,1997,13.0,282010.0
...,...,...,...,...,...
4195207,"Action, FPS, Shooter, Multiplayer, Singleplaye...",80.0,2004,0.0,80.0
4195208,"Action, FPS, Shooter, Multiplayer, Singleplaye...",80.0,2004,0.0,80.0
4195209,"Action, FPS, Shooter, Multiplayer, Singleplaye...",80.0,2004,0.0,80.0
4195210,"Action, FPS, Shooter, Multiplayer, Singleplaye...",80.0,2004,9.0,80.0


- Así observamos que la tabla resultante tendra un tamaño considerablemente menor y cada genero que se encuentre en estas lista, tendra el mismo tiempo de juego por año.

- **Por ejemplo:** el juego Quake live se lanzó en el 2010, y tiene en particular el genero de FPS y Shooter, este juego va a general el mismo playtime_forever tanto para FPS como para shooter.

In [44]:
steam_users.drop(columns=['item_name'], inplace=True)

tabla_endpoint_1 = steam_users.groupby(['release_year','genres']).sum().reset_index()

tabla_endpoint_1.drop(columns=['item_id','steam_id'], inplace=True)

## Exportar tabla consulta 1

In [57]:
path_endpoint_1 = os.path.join('..','data','clear','01_play_time_genre_data.csv.gz')

tabla_endpoint_1.to_csv(path_endpoint_1, 
                        index=False,
                        compression='gzip')

## Función play_time_genres

In [74]:
def play_time_genres(genero: str):
  path_endpoint_1 = os.path.join('..','data','clear','01_play_time_genre_data.csv.gz')
  try:
    tabla1 = pd.read_csv(path_endpoint_1)
    
    year_max =tabla1[tabla1['genres'].str.contains(genero)][['release_year','playtime_forever']].groupby('release_year').sum().idxmax().iloc[0]
    return year_max
  
  except Exception as e :
    return {f'Upps, tuvimos el siguiente error: {e}'}